## 1. Write a python program which searches all the product under a particular product from www.amazon.ins.

In [1]:
#import library
import selenium
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import os
import base64
import re

In [2]:
#set up the Chrome driver and open the webpage
driver=webdriver.Chrome()  
driver.maximize_window()
driver.get("https://www.amazon.in/")

In [3]:
User_input = input('Enter the Product you are interested in :')

Enter the Product you are interested in : guitar


In [4]:
# Finding search menu by xpath
Search = driver.find_element(By.XPATH, '//input[@id="twotabsearchtextbox"]')
# clearing any previous input in search bar
Search.clear()
# Feeding input specified by user to search menu through send keys
Search.send_keys(User_input)
# Finding Search button for clicking through xpath
Search_button=driver.find_element(By.XPATH, '//input[@id="nav-search-submit-button"]')
# Clicking search button
Search_button.click()

## 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [5]:
# Make Empty list to scrap data
Brand =[]
Product = []
Rating =[]
No_of_ratings =[]
Price =[]
Return =[]
Excepted_delivery =[]
Availability =[]
Other_details =[]

In [6]:
URL= []
# range(0,3) used to scrape three pages on website
for page in range(0,3):
    url=driver.find_elements(By.XPATH, "//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for i in url:
        URL.append(i.get_attribute('href'))
    time.sleep(2)
    # locating next page button and clicking
    Nxt_page=driver.find_element(By.XPATH, '//a[@class="s-pagination-item s-pagination-button"]')
    Nxt_page.click()
    time.sleep(3)

In [7]:
from tqdm import tqdm
for i in tqdm(URL):
    driver.get(i)
    time.sleep(2)
    
    # Extracting Brand Name via Xpath
    try:
        brand=driver.find_element(By.XPATH, '/html/body/div[2]/div/div[5]/div[3]/div[4]/div[50]/div/table/tbody/tr[1]/td[2]/span')
        Brand.append(brand.text) 
    except NoSuchElementException:
        Brand.append('-')
        
    time.sleep(1)
    
    # Extracting product name via Xpath
    try:
        product =driver.find_element(By.XPATH, '//span[@id="productTitle"]')
        Product.append(product.text)
    except NoSuchElementException:
        Product.append('-')
    time.sleep(1)
    
    # Extracting Rating via Xpath
    try:
        rating=driver.find_element(By.XPATH, '//span[@class="a-size-medium a-color-base"]')
        Rating.append(rating.text)
    except NoSuchElementException:
        Rating.append('-')
    time.sleep(1)
    
    # Extracting No of Ratings via Xpath
    try:
        rating_count=driver.find_element(By.XPATH, '//span[@id="acrCustomerReviewText"]')
        No_of_ratings.append(rating_count.text)
    except NoSuchElementException:
        No_of_ratings.append('-')
    time.sleep(1)
    
    # Extracting Price via Xpath
    try:
        price=driver.find_element(By.XPATH, '//span[@class="a-price-whole"]')
        Price.append(price.text)
    except NoSuchElementException:
        Price.append('-')
    time.sleep(1)
    
    # Extracting Return or exchange detail via Xpath
    try:
        replacement=driver.find_element(By.XPATH, '//span[@class="a-size-small a-color-link a-text-normal"]')
        Return.append(replacement.text)
    except NoSuchElementException:
        Return.append('-')
    time.sleep(1)
    
    # Extracting Availability via Xath
    try:
        availability=driver.find_element(By.XPATH, '//span[@class="a-size-medium a-color-success"]')
        Availability.append(availability.text)
    except NoSuchElementException:
        Availability.append('-')
    time.sleep(1)

100%|████████████████████████████████████████████████████████████████████████████████| 177/177 [39:59<00:00, 13.56s/it]


In [8]:
len(Brand),len(Product),len(Rating),len(No_of_ratings),len(Price),len(Return),len(Availability),len(URL)

(177, 177, 177, 177, 177, 177, 177, 177)

In [10]:
#creating DataFrame
Guitar=pd.DataFrame({
    'Brand':Brand[0:180],
    'Product':Product,
    'Rating':Rating,
    'No. of ratings':No_of_ratings,
    'Price':Price,
    'Return/Exchange':Return,
    'Availability':Availability,
    'URL':URL})
Guitar

,Brand,Product,Rating,No. of ratings,Price,Return/Exchange,Availability,URL
0,-,"Kadence rosewood Guitar Frontier Series, Elect...",4 out of 5,"1,263 ratings","6,299",Free Delivery,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Kadence,Kadence Frontier guitar with Online Guitar lea...,3.9 out of 5,"3,272 ratings","5,299",Free Delivery,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Henrix,Henrix 38C 38 Inch Cutaway Acoustic Guitar Wit...,3.8 out of 5,"2,366 ratings","3,299",Free Delivery,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,-,INTERN 41 inches Acoustic Guitar with truss ro...,3.9 out of 5,37 ratings,"3,490",Free Delivery,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,-,"Juârez Acoustic Guitar Kit, 38 Inch Cutaway, 0...",4 out of 5,"40,763 ratings","2,179",Free Delivery,In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...,...
172,Ibanez,Ibanez MD39C-NT 39 Inches Spruce Wood Cutaway ...,4.3 out of 5,21 ratings,"8,499",Free Delivery,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
173,-,INTERN 40 inches Acoustic Guitar with Pick-up ...,3.6 out of 5,60 ratings,"4,499",Free Delivery,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
174,-,"SG Musical Guitar Belt, Guitar 4 pc. Silicon P...",-,-,299,7 days Replacement,In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
175,-,Ketsicart White Pearl Guitar Binding Purfling ...,-,-,766,Pay on Delivery,-,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...


## 3.Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [5]:
#import library
import selenium
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import time
import os
import base64
import re

In [22]:
#set up the Chrome driver and open the webpage
driver=webdriver.Chrome()  
driver.maximize_window()
imageUrl='https://www.google.co.in/imghp'
driver.get(imageUrl)
search_list=['fruits','cars','Machine Learning','Guitar','Cakes']
target_dir = r"C:\Users\YMehta\Test\Images"

In [23]:
for name in search_list:
    print(name)
    driver.get(imageUrl)
    time.sleep(2)

    # Locate search box and search for images
    search_image = driver.find_element(By.XPATH, '//textarea[@class="gLFyf"]')
    search_image.send_keys(name)
    time.sleep(2)
    
    # Locate and click search button
    search_button = driver.find_element(By.XPATH, '//span[@class="z1asCe MZy1Rb"]')
    search_button.click()
    time.sleep(5)  # Allow time for the search results to load
    
    # Find image elements
    images = driver.find_elements(By.XPATH, "//img[@class='YQ4gaf']")
    time.sleep(3)
    
    img_urls = []
    for image in images:
        source = image.get_attribute('src')
        if source and (source.startswith('http') or source.startswith('data:image')):
            img_urls.append(source)
            
        
    for i in range(min(len(img_urls), 10)):
        print(f"Downloading {i+1} of 10 images")
        img_data = img_urls[i]
        if img_data.startswith('http'):
            response = requests.get(img_data)
            file_path = os.path.join(target_dir, f"{name}_{i+1}.jpg")
            with open(file_path, 'wb') as file:
                file.write(response.content)
        elif img_data.startswith('data:image'):
            header, encoded = img_data.split(',', 1)
            data = base64.b64decode(encoded)
            file_path = os.path.join(target_dir, f"{name}_{i+1}.jpg")
            with open(file_path, 'wb') as file:
                file.write(data)



fruits
cars
Machine Learning
Guitar
Cakes


In [24]:
driver.close()

## 4.Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [6]:
#set up the Chrome driver and open the webpage
driver=webdriver.Chrome()  
driver.maximize_window()
url = "https://www.flipkart.com/"
driver.get(url)
time.sleep(2)

In [7]:
#search for 'oneplus nord'
serch_bar = driver.find_element(By.XPATH,"//input[@class='Pke_EE']")
serch_bar.send_keys("Oneplus Nord")

In [140]:
#clicking on search button
srch_btn = driver.find_element(By.XPATH,"//button[@class='_2iLD__']")
srch_btn.click()
time.sleep(5)

In [141]:
Brand_titles=[]
Brand_Names = []
product_price=[]
ram=[]
RAM = []
rom=[]
ROM = []
Phone_display = []
Display = []
product_color=[]
Color = []
battery_size=[]
product_camera=[]
Camera=[]

In [142]:
def remove_text_inside_parentheses(text):
    return re.sub(r'\s*\([^)]*\)', '', text)

In [143]:
#fetching brand title and name
Brand_tags=driver.find_elements(By.XPATH,'//div[@class="KzDlHZ"]')
for i in Brand_tags:
    Brand_titles.append(i.text)
Brand_Names = [remove_text_inside_parentheses(item) for item in Brand_titles]

#fetching price
Brand_price=driver.find_elements(By.XPATH,'//div[@class="Nx9bqj _4b5DiR"]')
for i in Brand_price:
    product_price.append(i.text)

#fetching ram    
ram_p=driver.find_elements(By.XPATH,'//li[contains(text(), "RAM")]')
for i in ram_p:
    ram.append(i.text[0:9])
RAM = ram[: len(ram) - 5]

#fetching rom
rom_p=driver.find_elements(By.XPATH,'//li[contains(text(), "ROM")]')
for i in rom_p:
    rom.append(i.text[11:17])
ROM = rom[: len(rom) - 5]

#fetching phone display
display_p =driver.find_elements(By.XPATH,'//li[contains(text(), "Display")]')
for i in display_p:
    Phone_display.append(i.text[0:21])
Display = Phone_display[: len(Phone_display) - 5]

In [144]:
pattern = re.compile(r'\(([^,]+),')
def extract_color(text):
    match = re.search(pattern, text)
    return match.group(1)

In [145]:
#fetching color
color_p=driver.find_elements(By.XPATH,'//div[@class="KzDlHZ"]')
for i in color_p:
    product_color.append(i.text)
Color = [extract_color(item) for item in product_color]

#fetching battery size
battery_p=driver.find_elements(By.XPATH,'//li[contains(text(), "Battery")]')
for i in battery_p:
    battery_size.append(i.text[0:9])

#fetching camera
camera_p=driver.find_elements(By.XPATH,'//li[contains(text(), "Camera")]')
for i in camera_p:
    product_camera.append(i.text)
Camera  = product_camera[: len(product_camera) - 5]

In [148]:
#closing webdriver
driver.close()

In [147]:
#creating DataFrame
flipkart=pd.DataFrame({'Brand Names':Brand_Names,'Product Price':product_price,'Ram Size ( GB )':RAM,'Rom size':ROM,'Display Size': Display,'SmartPhone Color':Color,'SmartPhone Battery Size':battery_size,'SmartPhone Camera':Camera})
flipkart.to_csv('results.csv')
flipkart

,Brand Names,Product Price,Ram Size ( GB ),Rom size,Display Size,SmartPhone Color,SmartPhone Battery Size,SmartPhone Camera
0,OnePlus Nord CE 3 Lite 5G,"₹18,760",8 GB RAM,256 GB,17.07 cm (6.72 inch),Pastel Lime,5000 mAh,108MP Rear Camera
1,OnePlus Nord CE 3 Lite 5G,"₹16,889",8 GB RAM,128 GB,17.07 cm (6.72 inch),Chromatic Gray,5000 mAh,108MP Rear Camera
2,OnePlus Nord CE 3 Lite 5G,"₹18,600",8 GB RAM,256 GB,17.07 cm (6.72 inch),Chromatic Gray,5000 mAh,108MP Rear Camera
3,OnePlus Nord CE 3 Lite 5G,"₹16,843",8 GB RAM,128 GB,17.07 cm (6.72 inch),Pastel Lime,5000 mAh,108MP Rear Camera
4,OnePlus Nord CE4,"₹26,925",8 GB RAM,256 GB,17.02 cm (6.7 inch) D,Dark Chrome,5500 mAh,50MP Rear Camera
5,OnePlus Nord CE4 lite 5G,"₹19,672",8 GB RAM,128 GB,16.94 cm (6.67 inch),MEGA BLUE,5500 mAh,50MP Rear Camera
6,OnePlus Nord CE4,"₹24,540",8 GB RAM,128 GB,17.02 cm (6.7 inch) D,Celadon Marble,5500 mAh,50MP Rear Camera
7,OnePlus Nord CE4,"₹24,679",8 GB RAM,128 GB,17.02 cm (6.7 inch) D,Dark Chrome,5500 mAh,50MP Rear Camera
8,OnePlus Nord CE4 lite 5G,"₹19,785",8 GB RAM,128 GB,16.94 cm (6.67 inch),SUPER SILVER,5500 mAh,50MP Rear Camera
9,OnePlus Nord CE4,"₹26,990",8 GB RAM,256 GB,17.02 cm (6.7 inch) D,Celadon Marble,5500 mAh,50MP Rear Camera


## 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [156]:
#set up the Chrome driver and open the webpage
driver=webdriver.Chrome()  
driver.maximize_window()
driver.get("https://www.google.com/maps/")

In [157]:
#search for city
time.sleep(5)
search=driver.find_element(By.XPATH,'//input[@class="fontBodyMedium searchboxinput xiQnY "]')
search.send_keys("Pune")
search_button=driver.find_element(By.XPATH,'//button[@class="mL3xi"]')
search_button.click()
time.sleep(5)

In [158]:
lat=[]
long=[]

In [159]:
try:
    url_string = driver.current_url
    lat_long = re.findall(r'@(.*)data',url_string)
    if len(lat_long):
        lat_long_list = lat_long[0].split(",")
        if len(lat_long_list)>=2:
            lat.append(lat_long_list[0])
            long.append(lat_long_list[1])
except Exception as e:
    print("error:",str(e))

In [160]:
time.sleep(2)
driver.close()

In [161]:
Coordinates = pd.DataFrame({'Enter Location':"Pune",'latitude':lat,'longitude':long})
Coordinates

,Enter Location,latitude,longitude
0,Pune,18.5248706,73.6981469


## 6.Write a program to scrap all the available details of best gaming laptops from digit.in.

In [11]:
#set up the Chrome driver and open the webpage
driver=webdriver.Chrome()  
driver.maximize_window()
driver.get("https://www.digit.in/")

In [ ]:
# Clicking on top 10 option button in menu bar
driver.find_element(By.XPATH, '/html/body/div[1]/header/div/div[1]/div/div/nav/ul/li[3]/a').click()
time.sleep(3)

# Clicking on Laptop under table
driver.find_element(By.XPATH, '//div[@class="categoty_list"]/button[2]').click()

# Opening Best gaming laptops link
Gaming_lappy=driver.find_element(By.XPATH, '//div[@id="laptops"]/div[3]/a').get_attribute('href')
driver.get(Gaming_lappy)   
time.sleep(3)

## 7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [2]:
#set up the Chrome driver and open the webpage
driver=webdriver.Chrome()  
driver.maximize_window()
driver.get("https://www.forbes.com/")

In [3]:
#Try block to navigating to page all billionaires page
try:
    menu=driver.find_element(By.XPATH,"//div[@class='header__left']")
    menu.click()
    
    billionaires=driver.find_element(By.XPATH,"//li[@data-title='Billionaires']")
    billionaires.click()

    worldBillionaires=driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a')
    worldBillionaires.click()
    
except NoSuchElementException:
    #Except block if try block doesn't works 
    driver.get("https://www.forbes.com/billionaires/")

In [4]:
time.sleep(3)
rank=[]
name=[]
net_worth=[]
age=[]
citizenship=[]
source=[]
industry=[]

In [5]:
for page in range(0,14):
    data = driver.find_elements(By.XPATH,'//div[@class="Table_tableRow__lF_cY"]')
    
    for item in data:
#fetching rank
        try:
            b_rank = item.find_element(By.XPATH, './/div[@class="Table_rank__X4MKf"]').text
            rank.append(b_rank)
        except NoSuchElementException:
            rank.append("-")
#fetching name        
        try:
            b_name = item.find_element(By.XPATH, './/div[@class="Table_personName__Bus2E"]').text
            name.append(b_name)
        except NoSuchElementException:
            name.append("-")
#fetching net worth        
        try:
            b_networth = item.find_element(By.XPATH, './/div[@class="Table_finalWorth__UZA6k"]//span').text
            net_worth.append(b_networth)
        except NoSuchElementException:
            net_worth.append("-")
#fetching age        
        try:
            b_age = item.find_element(By.XPATH, './/div[4]').text
            age.append(b_age)
        except NoSuchElementException:
            age.append("-")
#fetching citizenship        
        try:
            b_citizenship = item.find_element(By.XPATH, './/div[5]').text
            citizenship.append(b_citizenship)
        except NoSuchElementException:
            citizenship.append("-")
 #fetching source       
        try:
            b_source = item.find_element(By.XPATH, './/div[6]').text
            source.append(b_source)
        except NoSuchElementException:
            source.append("-")
#fetching industry        
        try:
            b_industry = item.find_element(By.XPATH, './/div[7]').text
            industry.append(b_industry)
        except NoSuchElementException:
            industry.append("-")

In [8]:
#closing driver
driver.close()

In [6]:
#creating DtaFrame
billionaires = pd.DataFrame({
    'Rank': rank,
    'Name': name,
    'Net Worth': net_worth,
    'Age': age,
    'Citizenship': citizenship,
    'Source': source,
    'Industry': industry
})
billionaires

,Rank,Name,Net Worth,Age,Citizenship,Source,Industry
0,2.,Elon Musk,$195 B,52,United States,"Tesla, SpaceX",Automotive
1,3.,Jeff Bezos,$194 B,60,United States,Amazon,Technology
2,4.,Mark Zuckerberg,$177 B,39,United States,Facebook,Technology
3,5.,Larry Ellison,$141 B,79,United States,Oracle,Technology
4,6.,Warren Buffett,$133 B,93,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
2787,195.,Lei Jun,$10.9 B,Technology,-,-,-
2788,195.,Georg Schaeffler,$10.9 B,Automotive,-,-,-
2789,195.,Marcel Herrmann Telles & family,$10.9 B,Food & Beverage,-,-,-
2790,199.,David Velez & family,$10.8 B,Finance & Investments,-,-,-


## 8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [33]:
#set up the Chrome driver and open the webpage
url = "https://www.youtube.com/watch?v=X3paOmcrTjQ"
driver=webdriver.Chrome()  
driver.maximize_window()
driver.get(url)

In [49]:
Comments = []
Likes = []
Comment_posted_ago = []

In [50]:
for _ in range(30):
    driver.execute_script("window.scrollBy(0,100)")
    time.sleep(3)
#fetching comment    
    post_cooment = driver.find_elements(By.ID,"content-text")
    for i in post_cooment:
        Comments.append(i.text)
#fetching comment posted ago        
    post_time = driver.find_elements(By.XPATH,'//a[@class="yt-simple-endpoint style-scope ytd-comment-view-model"]')
    for i in post_time:
        Comment_posted_ago.append(i.text)
    Comment_posted_ago = Comment_posted_ago[0:1200]   
#fetching likes        
    post_upvotes = driver.find_elements(By.XPATH,'//span[@class="style-scope ytd-comment-engagement-bar"]')
    for i in post_upvotes:
        Likes.append(i.text)

In [51]:
#checking lenth
len(Comments),len(Likes),len(Comment_posted_ago)

(1200, 1200, 1200)

In [52]:
#closing webdriver
driver.close()

In [53]:
#creating DataFrame
youtube_video_analysis=pd.DataFrame({'Comments':Comments,'Comments Upvote':Likes,'Comment is POSTED':Comment_posted_ago})
youtube_video_analysis

,Comments,Comments Upvote,Comment is POSTED
0,Caltech Post Graduate Program In Data Science...,76,
1,Great video! As an established data scientist ...,942,
2,"So, i watched this video, and im even more con...",864,2 years ago (edited)
3,it was a great overview of all the process. i...,92,
4,This is really a very clear presentation of wh...,68,
...,...,...,...
1195,Great Video! Good pacing and information.,2,
1196,Great video..Thanks Simplilearn,,5 years ago
1197,Awesome video! Please let me know the Mathemat...,28,
1198,Do you have any questions on this topic? Pleas...,108,@CromaCampusOfficial


## 9. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [146]:
#set up the Chrome driver and open the webpage
driver = webdriver.Chrome()
driver.maximize_window()
try:

    driver.get("https://www.hostelworld.com/")

    loc = driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[1]/div[1]/div/div[2]/div[4]/div/div[2]/div/div[1]/div/div/div/input")

    loc.send_keys("london")
    search_btn = driver.find_element(By.XPATH,"/html/body/div[3]/div/div/div[2]/div[1]/div/div/div[4]/div/div[2]/div/div[5]/button")
    search_btn.click()

except Exception as e:

    driver.get(
        "https://www.hostelworld.com/s?q=London, England&country=England&city=London&type=city&id=3&from=2022-07-28&to=2022-07-31&guests=2&HostelNumber"
    )

In [147]:
time.sleep(3)

hostel_name = []
location = []
ratings = []
total_reviews = []
overall_reviews = []
private_price = []
dorm_price = []

In [148]:
#fetching hostel name
hostel = driver.find_elements(By.XPATH, '//div[@class="property-name"]//span')
try:
    for i in hostel:
        hostel_name.append(i.text)
except:
    hostel_name.append("-")

#fetching location
distance = driver.find_elements(By.XPATH, "//span[@class='distance-description']")
try:
    for i in distance:
        location.append(i.text)
except:
    location.append("-")

#fetching ratings
information = driver.find_elements(By.XPATH, "//span[@class='number']")
try:
    for i in information:
        ratings.append(i.text.split("\n")[0])
except:
    ratings.append("-")
ratings =ratings[0:30]

#fetching total reviews
reviews = driver.find_elements(By.XPATH, "//span[@class='keyword']")
try:
    for i in reviews:
        total_reviews.append(i.text)
except:
    total_reviews.append("-")
total_reviews = total_reviews[0:30]

#fetching overall reviews
review = driver.find_elements(By.XPATH, "//span[@class='left-margin']")
try:
    for i in review:
        overall_reviews.append(i.text)
except:
    overall_reviews.append("-")

#fetching private price
private = driver.find_elements(By.XPATH, "//div[@class='accommodation-price single-row']//strong")
try:
    for i in private:
        private_price.append(i.text.replace("\n", ""))
except:
    private_price.append("-")
private_price = private_price[::2]

#fetching dorm price
dorm = driver.find_elements(By.XPATH, "//div[@class='accommodation-price single-row']//strong")
for i in dorm:
    dorm_price.append(i.text.replace("\n", ""))
dorm_price = dorm_price[1::2]

In [149]:
#closing webdriver
driver.close()

In [152]:
#creating DataFrame
available_hostel = pd.DataFrame(
    {
        "Hostel Name": hostel_name[0:21],
        "Distance from City Center": location[0:21],
        "Ratings": ratings[0:21],
        "Total Review": total_reviews[0:21],
        "Overall Review": overall_reviews[0:21],
        "Private Price": private_price[0:21],
        "Dorm Price": dorm_price[0:21]
    }
)

In [153]:
available_hostel

,Hostel Name,Distance from City Center,Ratings,Total Review,Overall Review,Private Price,Dorm Price
0,Wombat's City Hostel London,- 3.6km from city centre,8.7,Fabulous,(16682),€256,€45
1,Palmers Lodge - Swiss Cottage,- 6.5km from city centre,7.7,Very Good,(16302),€97,€28
2,Urbany Hostel London,- 5.4km from city centre,9.3,Superb,(1349),€123,€40
3,Onefam Notting Hill,- 5.5km from city centre,8.7,Fabulous,(2876),€157,€48
4,Onefam Waterloo,- 0.7km from city centre,9.5,Superb,(404),€40,€116
5,NX London Hostel,- 6.1km from city centre,9.7,Superb,(2482),€36,€27
6,St Christopher's Village,- 1.8km from city centre,9.5,Superb,(13150),€38,€141
7,Clink261,- 3.2km from city centre,8.8,Fabulous,(715),€32,€26
8,Generator London,- 3km from city centre,7.7,Very Good,(8325),€119,€24
9,Astor Museum Inn,- 2.2km from city centre,8.2,Fabulous,(9558),€30,€21
